In [24]:
from collections import deque
import json
import os
import requests
import urllib.parse
import validators
from bs4 import BeautifulSoup

directory = "D:\deep_data\wiki_technicalTermLabeledData"

if not os.path.exists(directory):         # Ensures to make directory
    os.makedirs(directory)



q = deque()
set_depth = 4
base_url="https://en.wikipedia.org"
parent_url = "https://en.wikipedia.org/wiki/Wikipedia:Contents/Technology_and_applied_sciences"
q.append((parent_url,0))
checked=[]
repeated=[]
excpt = []
time_max = 0

count1 = count2 = count3 = count4 = 0

while(len(q)):
    front = q.popleft()                     # Basic BFS settings
    url = front[0]
    time = front[1]
    time_max = max(time_max, time)
    
    if(time_max>set_depth):
      q.appendleft((front,time))
      break
    
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    paragraphs = soup.find_all("p")
    print(url,len(paragraphs),time)
    count1+=1
    for j in range(len(paragraphs)):              # Inserting new links (according to given depth) to queue (BFS fashion)
      count2+=1
      if len(paragraphs[j].find_all("a"))!=0:
        list2=[]
        for i in paragraphs[j].find_all("a"):
          try:
            count3+=1
            if i.get("href") not in checked and validators.url(i.get("href")):
              checked.append(i.get("href"))
              count4+=1
              q.append((i.get("href"),time+1))
            elif base_url+i.get("href") not in checked and validators.url(base_url+i.get("href")):
              checked.append(base_url+i.get("href"))
              count4+=1
              q.append((base_url+i.get("href"),time+1))
            else:
              repeated.append(url)
              count4+=0
          except Exception:
            excpt.append([url])
            print("cought exception for ",url,i)

    texts = []
    labels = []
    count = 0
    for paragraph in paragraphs:          # Extrcating labeled and unlabeled data
      text = paragraph.get_text()
      texts.append(text)
      links = paragraph.find_all("a")
      label = [link.get_text() for link in links]
      labels.append(label)
      count+=1

    dict1 = {}                            # Saving
    for i in range(count):
      dict1[texts[i]] = labels[i]
    with open(os.path.join(directory,str(time)+urllib.parse.quote(url, safe="")+".json"),"w") as file:
      json.dump(dict1, file, indent="")
      

deque_list = list(q)      # Save the list to a JSON file
with open(os.path.join(directory,"deque_data.json"), "w") as file:
    json.dump(deque_list, file)
    

https://en.wikipedia.org/wiki/Wikipedia:Contents/Technology_and_applied_sciences 28 0
https://en.wikipedia.org/wiki/Human 95 1
https://en.wikipedia.org/wiki/Prehistory 35 1
https://en.wikipedia.org/wiki/Fire 34 1
https://en.wikipedia.org/wiki/Wheel 43 1
https://en.wikipedia.org/wiki/Printing_press 57 1
https://en.wikipedia.org/wiki/Internet 101 1
https://en.wikipedia.org/wiki/Communication 75 1
https://en.wikipedia.org/wiki/Weapon 34 1
https://en.wikipedia.org/wiki/Club_(weapon) 16 1
https://en.wikipedia.org/wiki/Nuclear_bomb 94 1
https://en.wikipedia.org/wiki/Applied_science 15 1
https://en.wikipedia.org/wiki/Natural_science 58 1
https://en.wikipedia.org/wiki/Engineering 91 1
https://en.wikipedia.org/wiki/Research_and_development 21 1
https://en.wikipedia.org/wiki/Technology 60 1
https://en.wikipedia.org/wiki/List_of_applied_sciences 2 1
https://en.wikipedia.org/wiki/Accelerating_change 25 1
https://en.wikipedia.org/wiki/Appropriate_technology 28 1
https://en.wikipedia.org/wiki/Diffus

In [22]:
print(len(checked),len(repeated))
print(count1,"level 1 urls + 1(for level0) + 1(for level2)")
print(count2,"sum(len(paragraphs of level0))")
print(count3,"sum(all links at level 1)")
print(count4,"sum of unchecked + checked links")

15891 11960
173 level 1 urls + 1(for level0) + 1(for level2)
6349 sum(len(paragraphs of level0))
27851 sum(all links at level 1)
15891 sum of unchecked + checked links
